In [1]:
import pandas as pd
import requests
import random
import time

In [2]:
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0",
    "Accept-Language": 'en-US,en;q=0.9',
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Referer": "https://tiki.vn/",
    "From": "",
    "af-ac-enc-dat": "",
    "x-api-source": "pc"
}

In [3]:
sub_category = 271966792
product_id = 184036446

In [4]:
URL = f"https://tiki.vn/api/v2/products/{product_id}"
PARAMS = {
}

In [5]:
response = requests.get(URL, headers=HEADERS, params=PARAMS)
data = response.json()

product_data = {
    'product_id': data['id'],
    'product_name': data.get('name', None),
    'product_url': data.get('short_url', None),
    'pricing_current': data.get('price', None),
    'pricing_original': data.get('original_price', None),
    'product_image_url': data.get('thumbnail_url', None),
    'inventory_status': data.get('inventory_status', None),
    'inventory_type': data.get('inventory_type', None),
    'created_date': data.get('day_ago_created', None),
    'quantity_sold': data.get('all_time_quantity_sold', None),
    'brand_id': data.get('brand', {}).get('id', None),
    'brand_name': data.get('brand', {}).get('name', None),
    'brand_slug': data.get('brand', {}).get('slug', None),
    'seller_id': data.get('current_seller', {}).get('id', 0) if data.get('current_seller') else 0,
    'seller_name': data.get('current_seller', {}).get('name', 0) if data.get('current_seller') else 0,
    'seller_link': data.get('current_seller', {}).get('link', 0) if data.get('current_seller') else 0,
    'seller_image_url': data.get('current_seller', {}).get('logo', 0) if data.get('current_seller') else 0
}

# sub_category ID
if data['categories']['is_leaf']:
    category_id = data['categories']['id']
else:
    category_id = data['breadcrumbs'][-2]['category_id']
product_data['category_id'] = category_id

In [6]:
product_df = pd.DataFrame([product_data],columns= ['product_id', 'product_name', 'product_url', 'pricing_current', 'pricing_original', 'product_image_url', 'inventory_status', 'inventory_type', 'created_date', 'quantity_sold', 'brand_id', 'brand_name', 'brand_slug', 'seller_id', 'seller_name', 'seller_link', 'seller_image_url', 'category_id'])
product_df

,product_id,product_name,product_url,pricing_current,pricing_original,product_image_url,inventory_status,inventory_type,created_date,quantity_sold,brand_id,brand_name,brand_slug,seller_id,seller_name,seller_link,seller_image_url,category_id
0,184036446,Apple iPhone 11,https://tiki.vn/product-p184036446.html?spid=3...,8650000,12990000,https://salt.tikicdn.com/cache/280x280/ts/prod...,available,instock,707,13990,17827,Apple,apple,1,Tiki Trading,https://tiki.vn/cua-hang/tiki-trading,21/ce/5c/b52d0b8576680dc3666474ae31b091ec.jpg,1795


In [7]:
URL = "https://tiki.vn/api/v2/reviews"
PARAMS = {"limit": 20, 
          "spid": sub_category, 
          "product_id": product_id}

In [8]:
feedback_data = []

# Get the total number of pages
response = requests.get(URL, headers=HEADERS, params=PARAMS)
data = response.json()
total_pages = data.get("paging", {}).get("last_page", 1)

# Fetch data from each page
for page in range(1, total_pages + 1):
    PARAMS["page"] = page
    response = requests.get(URL, headers=HEADERS, params=PARAMS)
    time.sleep(random.uniform(3.2, 8.7))
    data = response.json()

    stars = data.get("stars", {})
    OneStarCount = stars.get("1", {}).get("count", 0)
    TwoStarCount = stars.get("2", {}).get("count", 0)
    ThreeStarCount = stars.get("3", {}).get("count", 0)
    FourStarCount = stars.get("4", {}).get("count", 0)
    FiveStarCount = stars.get("5", {}).get("count", 0)
    reviews_count = data.get("reviews_count", 0)

    review_data = data.get("data", [])
    for review in review_data:
        review_id = review.get("id")
        review_title = review.get("title")
        review_content = review.get("content")
        review_upvote = review.get("thank_count", 0)
        review_rating = review.get("rating")
        review_created_at = review.get("created_at")
        reviewer = review.get("created_by", {})
        user_id = reviewer.get("id")
        username = reviewer.get("name")
        joined_time = reviewer.get("created_time")
        total_reviews = reviewer.get("contribute_info", {}).get("summary", {}).get("total_review", 0)
        total_upvotes = reviewer.get("contribute_info", {}).get("summary", {}).get("total_thank", 0)
        feedback_data.append([OneStarCount, TwoStarCount, ThreeStarCount, FourStarCount, FiveStarCount, reviews_count, review_id, review_title, review_content, review_upvote, review_rating, review_created_at, user_id, username, joined_time, total_reviews, total_upvotes])

feedback_df = pd.DataFrame(feedback_data, columns=["OneStarCount", "TwoStarCount", "ThreeStarCount", "FourStarCount", "FiveStarCount", "reviews_count", "review_id", "review_title", "review_content", "review_upvote", "review_rating", "review_created_at", "user_id", "username", "joined_time", "total_reviews", "total_upvotes"])

In [9]:
feedback_df

,OneStarCount,TwoStarCount,ThreeStarCount,FourStarCount,FiveStarCount,reviews_count,review_id,review_title,review_content,review_upvote,review_rating,review_created_at,user_id,username,joined_time,total_reviews,total_upvotes
0,61,13,32,183,2700,2989,19708302,Cực kì hài lòng,Đợt này mua tặng 2 cây trắng đen nhưng quên ch...,3,5,1704256818,5178168,Đại Hồng Bào,2017-04-10 14:05:29,273,369
1,61,13,32,183,2700,2989,19708307,Cực kì hài lòng,Đợt này mua tặng 2 cây trắng (giao từ Hanoi) đ...,2,5,1704256916,5178168,Đại Hồng Bào,2017-04-10 14:05:29,273,369
2,61,13,32,183,2700,2989,17356319,Cực kì hài lòng,"5* cho máy còn nguyên seal, hàng chính hãng VN...",184,5,1660043992,945720,muasach,2015-11-07 17:41:14,154,234
3,61,13,32,183,2700,2989,19694107,Rất không hài lòng,"Mua xài mới có 1 ngày mà đã lỗi, có cuộc gọi t...",1,1,1703434010,15052027,TRAN THUY,2019-11-20 17:08:29,39,1
4,61,13,32,183,2700,2989,18794713,Cực kì hài lòng,Chất lượng sản phẩm của Shop rất tốt đẹp full ...,20,5,1676287655,28830179,Minh Phương,2023-02-13 13:39:52,1,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2988,61,13,32,183,2700,2989,2804952,Ip11 màu đen,good,0,5,1577876061,335589,Huỳnh Hưng,2014-04-22 16:58:35,31,4
2989,61,13,32,183,2700,2989,17349937,Rất không hài lòng,(*) Đánh giá không tính điểm\nSáng mua giá 10....,0,1,1660014909,5982584,Lê Nghĩa,2017-10-28 11:20:36,58,6
2990,61,13,32,183,2700,2989,17371017,Rất không hài lòng,(*) Đánh giá không tính điểm\nTiki chăm sóc kh...,0,1,1660140662,15436788,QUYNH NGUYEN,2019-12-12 22:55:05,81,5
2991,61,13,32,183,2700,2989,17952552,Rất không hài lòng,(*) Đánh giá không tính điểm\nThứ nhất: J&T là...,0,1,1665676499,533759,Chủ Lò,2015-01-10 17:35:21,57,40
